CS594 Green Line
[Due on Oct19]Step1: Collect data on list of dispensaries from online : 
1. In Phase1, fields are as below:
>name
>street lnie 1,street line 2,city,state,zip code
>tel

2. As for weeds lists, we assume that the dispensaries have their own inventories like a CSV or .xls file containing the usage(recreational, medical, etc), storage(0-10^10 gram), unit prices for each weed items.

3. Pase2:
>image(TBD), open time


In [11]:
import io, time, json
import requests
from bs4 import BeautifulSoup
import re

In [14]:
def parse_page(html):
    """
    Parse the dispensary links on a single page of a websites.
    
    Args:
        html (string): String of HTML

    Returns:
        
    """
    soup = BeautifulSoup(html,'html.parser')
    urls = soup.find_all('a', itemprop="url")
    #use beautifulsoup CSS selectors to select and construct the review dict
    d_links=[]
    for url in urls:
        d_link = url.get('href')
        d_links.append(d_link)
    return d_links

In [33]:
def retrieve_html(url, parse_f=parse_page):
    """
    Return the raw HTML at the specified URL.
    """
    resp = requests.request('GET', url)
    print(resp)
    dispensary_links = parse_f(resp.text)
    print(dispensary_links)
    #use a list to store all dispensary links
    
    return resp.status_code, dispensary_links

In [16]:
#retrieve_html('https://wheresweed.com/illinois/chicago/marijuana-dispensaries')

In [17]:
#https://potguide.com/illinois/marijuana-dispensaries/chicago/
def parse_Chicago_dispensaries_page(html):
    """
    Parse the reviews on a single page of a restaurant.
    
    Args:
        html (string): String of HTML corresponding to a Yelp restaurant

    Returns:
        tuple(list, string): a tuple of two elements
            first element: list of dictionaries corresponding to the extracted review information
            second element: URL for the next page of reviews (or None if it is the last page)
    """
    soup = BeautifulSoup(html,'html.parser')

    divs = soup.find_all('div', "basic-listing")
    
    #use beautifulsoup CSS selectors to select and construct the review dict
    dispensaries = []
    for d in divs:
        dispensary={}
#         d_profile = "https://potguide.com/" + d.contents[1].contents[0].get('href')
        d_name = d.contents[1].contents[0].string
        d_add = d.contents[2].string.strip() + ", " + d.contents[4].string.strip()
        d_phone = d.contents[6].string.strip().lstrip(" Phone: ")
        d_website = d.contents[11].get("href")
        
        dispensary["name"] = d_name
#         dispensary["profile"] = d_profile
        dispensary["add"] = d_add
        dispensary["phone"] = d_phone
        dispensary["website"] = d_website
#         print(d.contents)
        dispensaries.append(dispensary)
        print("d_website", d_website)
    return dispensaries

# Stage 2 Some other webiste to collect dispensaries info

## 1 Work on new dispensary Json
TBD

In [150]:
# name, category, flavor, unit price, totals-in-storage
stat_code, dispensaries2 = retrieve_html("https://potguide.com/illinois/marijuana-dispensaries/chicago/", 
                              parse_Chicago_dispensaries_page)

d_website https://col-care.com/location/chicago?utm_source=PotGuide&sid=159225
d_website http://dispensary33.com/?utm_source=PotGuide&sid=159226
d_website http://greengatechicago.com/?utm_source=PotGuide&sid=499029
d_website https://www.maribisllc.com/?utm_source=PotGuide&sid=159240
d_website https://www.medmardispensary.com/?utm_source=PotGuide&sid=159242
d_website http://www.midwaydispensary.com/?utm_source=PotGuide&sid=159244
d_website https://missiondispensaries.com/chicago-il/?utm_source=PotGuide&sid=179810
d_website https://moderncann.com/?utm_source=PotGuide&sid=159247
d_website http://www.numed.com/chicago?utm_source=PotGuide&sid=584880
d_website http://theherbalcarecenter.com/?utm_source=PotGuide&sid=159266
d_website http://www.zenleafil.com/?utm_source=PotGuide&sid=159270


In [151]:
with open('dispensaries2.json', 'w') as fp:
    json.dump(dispensaries2, fp)

## 2 Work on the weed list in each dispensary
>1 numed is downloaded/copied directly from its page source
>
>2 moca has a xml menu list (parse and convert xml to json)
>
>structure is as below:
```json
{
    "id": 649090341,
    "name": "1:1 Balance Spray Tincture",
    "description": null,
    "tags": null,
    "product_type": "Edible",
    "vendor": "Cresco",
    "brand": null,
    "thc_percentage": "200.6",
    "cbd_percentage": "100.71",
    "special": false,
    "active": true,
    "strain_type": null,
    "thc_values": "200.6",
    "cbd_values": "100.71",
    "vendor_original": null,
    "menu_row_prices": [
      {
        "id": 771326494,
        "created_at": "2019-11-02T13:00:04.000-07:00",
        "updated_at": "2019-11-02T13:00:04.000-07:00",
        "menu_row_id": 649090341,
        "product_weight": "1",
        "product_weight_unit": "pack",
        "price": "30.0",
        "quantity": 99999,
        "original_name": "CC 1:1 Balance Spray Tincture",
        "version": 658,
        "menu_id": 245
      }
    ],
    "thc_range": "200.6",
    "cbd_range": "100.7"
  }
```

In [61]:
#https://moderncann.com/menu/
#1. manually copy down the <input> from the source page, then
#2. use below function to extract the info for different product
def parse_moca_menu(filename):
    """
    Parse the menu on a single page of moderncannabis.com
    
    Args:
        filename (string): filename for a file storing string of xml corresponding to moca product menu

    Returns:
        tuple(list, string): a tuple of two elements
            first element: list of dictionaries corresponding to the extracted review information
            second element: URL for the next page of reviews (or None if it is the last page)
    """
    with open('menus/moca_menu.xml') as fd:
        soup = BeautifulSoup(fd, 'html.parser')
    
    inputs = soup.find_all('input')
    print("len(inputs)", len(inputs))
    #use beautifulsoup CSS selectors to select and construct the review dict
    products = []
    # every 16 items, we generate a new product
    for i in range(int(len(inputs)/16)):
        product={}
        product["id"] = inputs[i*16].get("id").split("_")[1]
        product["name"] = inputs[i*16+1].get("value")
        product["description"] = None
        product["tags"] = None
        product["product_type"] = inputs[i*16].get("value")
        product["vendor"] = inputs[i*16+8].get("value")
        product["thc_percentage"] = inputs[i*16+3].get("value")
        product["cbd_percentage"] = inputs[i*16+6].get("value")
        product["special"] = False
        product["active"] = True
        product["strain_type"] = None
        product["thc_values"] = inputs[i*16+3].get("value")
        product["cbd_values"] = inputs[i*16+6].get("value")
        product["strain_type"] = None
        product["vendor_original"] = None
        product["menu_row_prices"] = {}
        product["menu_row_prices"]["id"] = inputs[i*16].get("id").split("_")[2]
        product["menu_row_prices"]["created_at"] = "2019-11-02T13:00:04.000-07:00"
        product["menu_row_prices"]["updated_at"] = "2019-11-02T13:00:04.000-07:00"
        product["menu_row_prices"]["menu_row_id"] = inputs[i*16].get("id").split("_")[2]
        product["menu_row_prices"]["updated_at"] = "2019-11-02T13:00:04.000-07:00"
        product["menu_row_prices"]["product_weight"] = inputs[i*16+9].get("value")
        product["menu_row_prices"]["product_weight_unit"] = "g"
        product["menu_row_prices"]["price"] = inputs[i*16+15].get("value")
        product["menu_row_prices"]["quantity"] = 0 if inputs[i*16+15].get("value") == "0" else 99999
        product["menu_row_prices"]["original_name"] = None
        product["menu_row_prices"]["version"] = 600
        product["menu_row_prices"]["menu_id"] = 999
        product["thc_range"] = inputs[i*16+3].get("value")
        product["cbd_range"] = inputs[i*16+6].get("value")
        products.append(product)
        print("product", product)
    return products

In [50]:
# stat_code, products = retrieve_html("https://moderncann.com/menu/", parse_moca_page)
retrieve_html("https://moderncann.com/menu/", parse_moca_page)

<Response [403]>
[]
[]


(403, [])

In [63]:
moca_products = parse_moca_menu("menus/moca_menu.xml")
with open('menus/moca_menu.json', 'w') as fp:
    json.dump(moca_products, fp)

len(inputs) 4688
product {'id': '4821', 'name': 'ASH CATCHER', 'description': None, 'tags': None, 'product_type': 'ASH CATCHERS', 'vendor': 'HSW', 'thc_percentage': '0.00', 'cbd_percentage': '0.00', 'special': False, 'active': True, 'strain_type': None, 'thc_values': '0.00', 'cbd_values': '0.00', 'vendor_original': None, 'menu_row_prices': {'id': 'group', 'created_at': '2019-11-02T13:00:04.000-07:00', 'updated_at': '2019-11-02T13:00:04.000-07:00', 'menu_row_id': 'group', 'product_weight': '0', 'product_weight_unit': 'g', 'price': '15', 'quantity': 99999, 'original_name': None, 'version': 600, 'menu_id': 999}, 'thc_range': '0.00', 'cbd_range': '0.00'}
product {'id': '3986', 'name': 'YOCAN STIX ATOMIZER', 'description': None, 'tags': None, 'product_type': 'ATOMIZERS', 'vendor': 'GOT VA', 'thc_percentage': '0.00', 'cbd_percentage': '0.00', 'special': False, 'active': True, 'strain_type': None, 'thc_values': '0.00', 'cbd_values': '0.00', 'vendor_original': None, 'menu_row_prices': {'id': '